# #30DayChartChallenge
**5th edition - April 2025**

In [ ]:
# Import Libraries for charting
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from owslib.wfs import WebFeatureService

# Initialize the map
m = geemap.Map()

## Retrieve WFS ICF Info

In [ ]:
url_wfs = "https://geoserver.icf.gob.hn/icfpub/wfs"
wfs = WebFeatureService(url_wfs, version="1.1.0")

# Get the list of available layers
layers = wfs.contents
# print("Available layers:")
# for layer in layers:
#     print(layer)

# 9th of April 2025
**Day 8: Distribution | Diverging**


In [ ]:
# Define the gee data
modis_lst_coll = (
    ee.ImageCollection("MODIS/061/MOD21A1D")
    .select("LST_1KM")
)


# Define the region of interest (ROI)
layer = "icfpub:limite_departamentos_gp"
response = wfs.getfeature(typename=layer, outputFormat="application/json")
gdf = gpd.read_file(response)
ee_roi = geemap.gdf_to_ee(gdf)
ee_roi_ext = ee_roi.geometry().bounds()


In [ ]:
# Clip the collection to the region of interest
coll_clipped = (
    modis_lst_coll
    .filterBounds(ee_roi_ext)
    .map(lambda img: img.clipToCollection(ee_roi))
)

# Convert to Celsius
def kelvin_to_celsius(img):
    return img.multiply(0.02).subtract(273.15).copyProperties(img, ['system:time_start'])

coll_celsius = coll_clipped.map(kelvin_to_celsius)

# Calculate annual average
def get_annual_mean(year):
    yearly_col = coll_celsius.filter(ee.Filter.calendarRange(year, year, 'year'))
    mean_img = yearly_col.mean().clipToCollection(ee_roi)
    return mean_img.set('year', year)

# Get the annual mean images
years = ee.List.sequence(2000, 2025)
annual_means = ee.ImageCollection(years.map(get_annual_mean))

# Historial Anual mean
historical_mean = annual_means.mean()

In [ ]:
# Función para restar el promedio histórico a cada año
def calculate_deviation(year_img):
    deviation = year_img.subtract(historical_mean).rename('temp_deviation')
    return deviation.set('year', year_img.get('year'))

# Aplicar a la colección anual
deviations = annual_means.map(calculate_deviation)

In [ ]:
# Reducir región (Honduras) a un valor medio por año
stats_list = []
for year_img in deviations.toList(deviations.size()).getInfo():
    year = year_img['properties']['year']
    mean_dev = ee.Image(year_img).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_roi,
        scale=1000  # Resolución MODIS
    ).get('temp_deviation').getInfo()
    stats_list.append({'year': year, 'deviation': mean_dev})

# Convertir a DataFrame
df = pd.DataFrame(stats_list).sort_values('year')